In [1]:
using Parameters
using BasisMatrices
using LaTeXStrings
using Plots; pyplot();
using QuantEcon
using FastGaussQuadrature

In [4]:
@with_kw immutable KrusselSmithAdjCost
    β::Float64                            # discount factor
    δ::Float64                            # depreciation of capital
    α::Float64                            # capital share
    γ::Float64                            # RRA
    ρz::Float64                           # AR coefficient on individual productivity
    σz::Float64                           # std dev of innovation on individual productivity
    ρA::Float64                           # AR coefficient on productivity
    σA::Float64                           # std dev of innovation of productivity process
    F::Float64                            # Fixed cost of adjustment    
end

@with_kw struct Household

    model::KrusselSmithAdjCost

    # Solution

    c::Vector{Float64}                    # coefficients on approximate value function
    cE::Vector{Float64}                   # coefficients on approximate expected value function

    Nk::Int64                             # number of nodes for capital
    Nz::Int64                             # number of nodes for individual productivity
    
    gridk::Vector{Float64}                # grid for capital
    gridz::Vector{Float64}                # grid for individual productivity

    s::Matrix{Float64}                    # full state space
    Ns::Int64                             # number of states

    basis::Basis                          # basis of approximate function
    bs::BasisMatrix                       # basis direct matrix
    Φ::SparseMatrixCSC                    # basis expanded matrix
    ΦE::SparseMatrixCSC                   # cE = ΦE * c
end

function Household(model, Nk = 45, Nz = 30)
    
    gridk = 500*linspace(0, 1, Nk) .^ 2.5
    gridz = linspace(-5*√(1/(1 - model.ρz^2))*model.σz, 5*√(1/(1 - model.ρz^2))*model.σz, Nz)
    
    basis = Basis(SplineParams(gridk, 0, 3), SplineParams(gridz, 0, 3))
    
    # the basis functions might need additional nodes at the boundaries
    s, (gridk, gridz) = nodes(basis)
    Ns, Na, Ny = size(s, 1), size(gridk, 1), size(gridz, 1)
    
    bs = BasisMatrix(basis, Direct())
    Φ = convert(Expanded, bs).vals[1]
    
    (e, w) = gausshermite(50)
    e = model.σz * e
    ΦE = spzeros(Ns, Ns)
    
    for j in 1:length(e)
        zprime = max.(min.(model.ρz*s[:,2] + e[j], maximum(gridz)), minimum(gridz))
        ΦE += w[j] * BasisMatrix(basis, Expanded(), hcat(s[:, 1], zprime)).vals[1]
    end
    
    ΦE = Φ \ ΦE
    
    return Household(model, zeros(Ns), zeros(Ns), Nk, Nz, gridk, gridz, s, Ns, basis, bs, Φ, ΦE)
end

Household

In [5]:
model = KrusselSmithAdjCost(β = 0.98, δ = 0.025, α = 1/3, γ = 2, ρz = 0.98, σz = 0.1, ρA = 0.95, σA = 0.007, F = 0);
hh = Household(model);

LoadError: [91mMethodError: no method matching A_ldiv_B!(::Base.SparseArrays.UMFPACK.UmfpackLU{Float64,Int64}, ::SparseMatrixCSC{Float64,Int64})[0m
Closest candidates are:
  A_ldiv_B!([91m::Base.LinAlg.LU{T,Tridiagonal{T}}[39m, ::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T) where T at linalg\lu.jl:413
  A_ldiv_B!([91m::Diagonal{T}[39m, ::AbstractArray{T,2}) where T at linalg\diagonal.jl:253
  A_ldiv_B!([91m::Union{Base.LinAlg.AbstractTriangular, Bidiagonal}[39m, ::AbstractArray{T,2} where T) at linalg\bidiag.jl:511
  ...[39m

In [26]:
Nk = 45; Nz = 30   
gridk = 500*linspace(0, 1, Nk) .^ 2.5
    gridz = linspace(-5*√(1/(1 - model.ρz^2))*model.σz, 5*√(1/(1 - model.ρz^2))*model.σz, Nz)
    
    basis = Basis(SplineParams(gridk, 0, 3), SplineParams(gridz, 0, 3))
    
    # the basis functions might need additional nodes at the boundaries
    s, (gridk, gridz) = nodes(basis)
    Ns, Na, Ny = size(s, 1), size(gridk, 1), size(gridz, 1)
    
    bs = BasisMatrix(basis, Direct())
    Φ = convert(Expanded, bs).vals[1]
    
    (e, w) = gausshermite(50)
    e = model.σz * e
    ΦE = spzeros(Ns, Ns)
    
    for j in 1:length(e)
        zprime = max.(min.(model.ρz*s[:,2] + e[j], maximum(gridz)), minimum(gridz))
        ΦE += w[j] * BasisMatrix(basis, Expanded(), hcat(s[:, 1], zprime)).vals[1]
    end

In [107]:
convert(SparseMatrixCSC, full(Φ) \ full(ΦE))

1504×1504 SparseMatrixCSC{Float64,Int64} with 2061344 stored entries:
  [1   ,    1]  =  0.820109
  [2   ,    1]  =  -1.32353e-16
  [3   ,    1]  =  6.10613e-17
  [4   ,    1]  =  -2.48283e-17
  [5   ,    1]  =  8.83069e-18
  [6   ,    1]  =  -2.91738e-18
  [7   ,    1]  =  9.22095e-19
  [8   ,    1]  =  -2.83079e-19
  [9   ,    1]  =  8.51384e-20
  [10  ,    1]  =  -2.52196e-20
  ⋮
  [1494, 1504]  =  1.76892e-21
  [1495, 1504]  =  -6.74216e-21
  [1496, 1504]  =  2.56826e-20
  [1497, 1504]  =  -9.77785e-20
  [1498, 1504]  =  3.72069e-19
  [1499, 1504]  =  -1.41511e-18
  [1500, 1504]  =  5.37965e-18
  [1501, 1504]  =  -2.04421e-17
  [1502, 1504]  =  7.76451e-17
  [1503, 1504]  =  -1.66362e-16
  [1504, 1504]  =  0.820109